In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyLDAvis.gensim
import re
import nltk
from nltk.stem.porter import *
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import gensim
from gensim import corpora, models
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from lda import guidedlda as glda
import numpy as np
%matplotlib inline


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_pickle("../ETL/data.pkl")
review_pd = pd.read_pickle("../ETL/review_pd.pkl")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
data.head()


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,name,seller,price,original_price,lectures,hours,level,enrollment,rating,...,downloadable_resources,last_update_date,instructor_title,instructor_job_title,instructor_info,instructor_rating,instructor_no_reviews,instructor_no_students,instructor_no_courses,review_index
0,1,the-complete-sql-bootcamp/,Jose Portilla,25.99,179.99,83,9.0,All Levels,342288.0,4.7,...,14.0,2020-10-01,Jose Portilla,"Head of Data Science, Pierian Data Inc.",Jose Marcial Portilla has a BS and MS in Mec...,4.6,712729.0,2179902.0,31.0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
1,2,tableau10/,Kirill Eremenko SuperDataScience Team,16.99,124.99,82,9.0,All Levels,231631.0,4.6,...,6.0,2021-02-01,Ligency Team,Helping Data Scientists Succeed,"Hi there, We are the Ligency PR and Marketing ...",4.5,429183.0,1472140.0,48.0,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]"
2,3,powerbi-complete-introduction/,Manuel Lorenz Academind by Maximilian Schwarz...,12.99,94.99,275,23.5,Beginner,148077.0,4.6,...,26.0,2020-11-01,Maximilian Schwarzmüller,Professional Web Developer and Instructor,Experience as (Web) Developer Starting out at ...,4.6,564184.0,1335299.0,35.0,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]"
3,4,microsoft-power-bi-up-running-with-power-bi-de...,Maven Analytics Chris Dutton Aaron Parry,19.99,139.99,117,11.0,All Levels,109303.0,4.6,...,14.0,2021-02-01,Aaron Parry,Professional Business Intelligence Trainer & C...,Aaron is a professional analytics consultant a...,4.6,34377.0,438163.0,3.0,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]"
4,5,the-business-intelligence-analyst-course-2018/,365 Careers,12.99,94.99,413,20.5,All Levels,136869.0,4.5,...,151.0,2020-11-01,365 Careers,Creating opportunities for Business & Finance ...,365 Careers is the #1 best-selling provider of...,4.5,385370.0,1322640.0,68.0,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]"


In [4]:
print(data.columns)


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['id', 'name', 'seller', 'price', 'original_price', 'lectures', 'hours',
       'level', 'enrollment', 'rating', 'num_ratings', 'five_stars',
       'four_stars', 'three_stars', 'two_stars', 'description',
       'hours_ondemand_video', 'downloadable_resources', 'last_update_date',
       'instructor_title', 'instructor_job_title', 'instructor_info',
       'instructor_rating', 'instructor_no_reviews', 'instructor_no_students',
       'instructor_no_courses', 'review_index'],
      dtype='object')


In [5]:
def preprocess_sent(text):
    """
    Get sentence level preprocessed data from raw texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    # Add missing delimiter: xxxThis is a --> xxx.
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', text)  # before lower case
    # Make words all lower case.
    s = s.lower()
    # Remove < and >
    s = re.sub(r'&gt|&lt', ' ', s)
    # Remove letters that repeat more than 2 time.
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # Remove non-word characters that repeat more than 1 time.
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # Use string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # Remove words in parenthesis, which are assumed less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # [.?!]xxx --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    return s.strip()


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# define stemmer

p_stemmer = PorterStemmer()
# create stop words list
stop_words = (list(
    set(get_stop_words('en'))
    |set(get_stop_words('es'))
    |set(get_stop_words('de'))
    |set(get_stop_words('it'))
    |set(get_stop_words('ca'))
    |set(get_stop_words('pt'))
    |set(get_stop_words('pl'))
    |set(get_stop_words('da'))
    |set(get_stop_words('ru'))
    |set(get_stop_words('sv'))
    |set(get_stop_words('sk'))
    |set(get_stop_words('nl'))
    |set(["course","data"])
))

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed documents
    """
    if not s:
        return []
    w_list = word_tokenize(s)
    w_list = [word for word in w_list if word.isalpha()]
    w_list = [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']
    w_list = [word for word in w_list if word not in stop_words]
    w_list = [p_stemmer.stem(word) for word in w_list]

    return w_list


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
import codecs
def lda_process(token_lists,output):
    """
    # Normal LDA without seeding
    :param token_lists:
    :return:
    """
    text_lists_clean = [ " ".join(tl) for tl in token_lists ]
    vocab=list(set(word_tokenize(" ".join(text_lists_clean))))
    vectorizer = CountVectorizer(ngram_range=(1,1),vocabulary=vocab)
    X = vectorizer.fit_transform(text_lists_clean)
    word2id = dict((v, idx) for idx, v in enumerate(vocab))
    # from lda import glda_datasets as gldad
    model = glda.GuidedLDA(n_topics=8, n_iter=1000, random_state=7, refresh=20)
    model.fit(X)
    topic_word = model.topic_word_
    n_top_words = 8
    f = codecs.open(output+"_topic.txt", "w", "utf-8")
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        f.write('Topic {}: {} \n'.format(i, ' '.join(topic_words)))
    f.close()

    doc_topic = pd.DataFrame(model.doc_topic_)
    doc_topic.to_csv(output+"_doc_topic.csv")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# LDA for course description
data["description"].fillna(" ", inplace=True)
docs = data["description"].to_list()
# print(len(docs), "course descriptions are collected.")

text_lists = [preprocess_sent(text) for text in docs]
token_lists = [preprocess_word(text) for text in text_lists]
lda_process(token_lists,"description")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:lda:n_documents: 1009
INFO:lda:vocab_size: 8342
INFO:lda:n_words: 64214
INFO:lda:n_topics: 8
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -679531
INFO:lda:<20> log likelihood: -491894
INFO:lda:<40> log likelihood: -480335
INFO:lda:<60> log likelihood: -474530
INFO:lda:<80> log likelihood: -472094
INFO:lda:<100> log likelihood: -470101
INFO:lda:<120> log likelihood: -467892
INFO:lda:<140> log likelihood: -466022
INFO:lda:<160> log likelihood: -464709
INFO:lda:<180> log likelihood: -464565
INFO:lda:<200> log likelihood: -464209
INFO:lda:<220> log likelihood: -463859
INFO:lda:<240> log likelihood: -463305
INFO:lda:<2

In [10]:
# LDA for instructor information
data["instructor_info"].fillna(" ", inplace=True)
docs = data["instructor_info"].to_list()

text_lists = [preprocess_sent(text) for text in docs]
token_lists = [preprocess_word(text) for text in text_lists]
lda_process(token_lists,"instructor_info")


C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:lda:n_documents: 1009
INFO:lda:vocab_size: 3771
INFO:lda:n_words: 26981
INFO:lda:n_topics: 8
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -291102
INFO:lda:<20> log likelihood: -195083
INFO:lda:<40> log likelihood: -190382
INFO:lda:<60> log likelihood: -187529
INFO:lda:<80> log likelihood: -186241
INFO:lda:<100> log likelihood: -185322
INFO:lda:<120> log likelihood: -185030
INFO:lda:<140> log likelihood: -184731
INFO:lda:<160> log likelihood: -184347
INFO:lda:<180> log likelihood: -183883
INFO:lda:<200> log likelihood: -183709
INFO:lda:<220> log likelihood: -183494
INFO:lda:<240> log likelihood: -183301
INFO:lda:<2

In [38]:
def glda_process(token_lists,seed_topic_list,output):
    """
    # guided lda
    :param token_lists:
    :return:
    """
    text_lists_clean = [ " ".join(tl) for tl in token_lists ]
    vocab=list(set(word_tokenize(" ".join(text_lists_clean))))
    vectorizer = CountVectorizer(ngram_range=(1,1),vocabulary=vocab)
    X = vectorizer.fit_transform(text_lists_clean)
    word2id = dict((v, idx) for idx, v in enumerate(vocab))
    # from lda import glda_datasets as gldad
    model = glda.GuidedLDA(n_topics=20, n_iter=1000, random_state=7, refresh=20)

    seed_topics = {}
    for t_id, st in enumerate(seed_topic_list):
        for word in st:
            seed_topics[word2id[word]] = t_id
    model.fit(X, seed_topics=seed_topics, seed_confidence=100)
    topic_word = model.topic_word_
    n_top_words = 8
    f = codecs.open(output+"_topic_guidedlda.txt", "w", "utf-8")
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        f.write('Topic {}: {} \n'.format(i, ' '.join(topic_words)))
    f.close()

    doc_topic = pd.DataFrame(model.doc_topic_)
    doc_topic.to_csv(output+"_doc_topic_guidedlda.csv")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# LDA for review_pd
review_pd["review_text"].fillna(" ", inplace=True)
docs = review_pd["review_text"].to_list()

text_lists = [preprocess_sent(text) for text in docs]
token_lists = [preprocess_word(text) for text in text_lists]
lda_process(token_lists,"review")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:lda:n_documents: 8720
INFO:lda:vocab_size: 9882
INFO:lda:n_words: 53278
INFO:lda:n_topics: 8
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -679557
INFO:lda:<20> log likelihood: -436790
INFO:lda:<40> log likelihood: -415952
INFO:lda:<60> log likelihood: -411472
INFO:lda:<80> log likelihood: -408671
INFO:lda:<100> log likelihood: -407387
INFO:lda:<120> log likelihood: -407135
INFO:lda:<140> log likelihood: -407075
INFO:lda:<160> log likelihood: -406566
INFO:lda:<180> log likelihood: -406523
INFO:lda:<200> log likelihood: -406434
INFO:lda:<220> log likelihood: -406101
INFO:lda:<240> log likelihood: -406026
INFO:lda:<2

In [39]:
review_pd["review_text"].fillna(" ", inplace=True)
docs = review_pd["review_text"].to_list()
seed_topic_list = [['curiosity', 'interest', 'discover', 'enthusiastic', 'question'],
                   ['knowledge', 'wisdom', 'job', 'skills', 'demand'],
                   ['credit', 'certificate', 'completion'],
                   ['truth', 'genuine', 'goal', 'effort', 'challenge', 'time']]

add_texts = [" ".join(l)  for l in seed_topic_list]
docs.extend(add_texts)
text_lists = [preprocess_sent(text) for text in docs]
token_lists = [preprocess_word(text) for text in text_lists]
seed_topic_list = token_lists[-4:]
glda_process(token_lists,seed_topic_list,"review")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:lda:n_documents: 8724
INFO:lda:vocab_size: 9883
INFO:lda:n_words: 53295
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -745381
INFO:lda:<20> log likelihood: -435103
INFO:lda:<40> log likelihood: -420962
INFO:lda:<60> log likelihood: -416393
INFO:lda:<80> log likelihood: -414301
INFO:lda:<100> log likelihood: -413365
INFO:lda:<120> log likelihood: -412423
INFO:lda:<140> log likelihood: -411643
INFO:lda:<160> log likelihood: -411416
INFO:lda:<180> log likelihood: -411041
INFO:lda:<200> log likelihood: -410721
INFO:lda:<220> log likelihood: -410365
INFO:lda:<240> log likelihood: -410190
INFO:lda:<

In [37]:
seed_topic_list

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['curios', 'interest', 'discov', 'question'],
 ['knowledg', 'wisdom', 'job', 'skill', 'demand'],
 ['credit', 'certif', 'complet'],
 ['truth', 'goal', 'effort', 'challeng', 'time']]

In [15]:
data.drop(['id', 'description','instructor_info'], axis=1,inplace = True)
data.to_csv("final_data.csv")

C:\Users\zhipe\Anaconda3\envs\glda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
